VGG 是使用块的深度神经网络  意图探索传统卷积神经网络有无一个比较规范化的模板
> 经典卷积神经网络的基本组成部分是下面的这个序列：
* 带填充以保持分辨率的卷积层；  
* 非线性激活函数，如ReLU；  
* 汇聚层，如最大汇聚层。
作者抽象出来了一个VGG块，并且通过循环使用这种VGG块达到加深网络的目的  

一个VGG块的组成：  
* 数个3x3的卷积，padding为1
* 每个卷积后面加个ReLu激活函数
* 最后出去一个Max poolig（stride为2的最大池化）

In [1]:
import torch
from torch import nn
from d2l import torch as d2l


def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels,
                                kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
    return nn.Sequential(*layers)

原始VGG网络有5个卷积块，其中前两个块各有一个卷积层，后三个块各包含两个卷积层。 第一个模块有64个输出通道，每个后续模块将输出通道数量翻倍，直到该数字达到512。由于该网络使用8个卷积层和3个全连接层，因此它通常被称为VGG-11  
> 注意，我们这边限制的都是通道数，而不是tensor的(w, h)  
> 由于max pooling的kernel = 2， stride = 2，所以操作之后肯定w和h会各自降低为原来的一半

In [ ]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

def vgg(conv_arch):
    conv_blks = []
    in_channels = 1
    # 卷积层部分
    # 将不同输入输出的vgg blocks嵌套进入一个大的List里
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels

    return nn.Sequential(
        *conv_blks, nn.Flatten(),
        # 全连接层部分
        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(), nn.Dropout(0.5),
        nn.Linear(4096, 10))

net = vgg(conv_arch)

In [ ]:
X = torch.randn(size=(1, 1, 224, 224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__,'output shape:\t',X.shape)

In [ ]:
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
# 所有的通道都除以4之后得到的VGG-11来做实验
net = vgg(small_conv_arch)

lr, num_epochs, batch_size = 0.05, 10, 128
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())